In [ ]:
# mount drive, install kaggle, download the data, and unzip
from google.colab import drive
drive.mount('/content/gdrive')
!pip install -q kaggle
%mkdir /root/.kaggle
%cp /content/gdrive/My\ Drive/CMU11785-HW2P2/kaggle.json  /root/.kaggle/
%cd /
!kaggle datasets download -d cmu11785/20fall-hw2p2
!unzip -q 20fall-hw2p2.zip -d data

Mounted at /content/gdrive
/
 98% 1.32G/1.35G [00:18<00:00, 110MB/s]
100% 1.35G/1.35G [00:18<00:00, 78.3MB/s]


In [ ]:
# change directory to hw2_p2
%cd /content/gdrive/My\ Drive/CMU11785-HW2P2/

/content/gdrive/My Drive/CMU11785-HW2P2


In [ ]:
# importing the packages
import os
import numpy as np
from PIL import Image
import time
import datetime
import torch.optim as optim
import torch
from torchvision import transforms, datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.metrics import roc_auc_score

In [ ]:
#smallest ResNET Block
class baseBlock(torch.nn.Module):
  def __init__(self,in_channel,out_channel,stride=1,shortcut=None):
      super(baseBlock,self).__init__()
      self.conv1 = torch.nn.Conv2d(in_channel,out_channel,stride=stride,kernel_size=3,padding=1)
      self.bn1   = torch.nn.BatchNorm2d(out_channel)
      self.conv2 = torch.nn.Conv2d(out_channel,out_channel,stride=1,kernel_size=3,padding=1)
      self.bn2   = torch.nn.BatchNorm2d(out_channel)
      self.shortcut = shortcut

  def forward(self,x):
      output = F.relu(self.bn1(self.conv1(x)))
      #print(output.shape)
      output = self.bn2(self.conv2(output))
      #print(output.shape)
      if self.shortcut is not None:
        output += self.shortcut(x)
      output = F.relu(output)
      #print(output.shape)
      return output


class ResNet(torch.nn.Module):
  def __init__(self,num_layers,classes=10,feats=512):
      super(ResNet,self).__init__()
      self.conv1 = nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1)
      self.bn1   = nn.BatchNorm2d(64)
      self.input_planes = 64
      self.layer1 = self._layer(64,  num_layers[0],stride=2)
      self.layer2 = self._layer(128,  num_layers[1],stride=2)
      self.layer3 = self._layer(256,  num_layers[2],stride=2)
      self.layer4 = self._layer(feats,num_layers[3],stride=2)
      self.avgPool = nn.AdaptiveAvgPool2d((2))
      self.fc  =  torch.nn.Linear(feats*2*2,classes)
  
  def _layer(self,planes,num_layers,stride=1):
      netLayers =[]
      shortcut = None
      if stride !=1 or self.input_planes != planes:
        shortcut = torch.nn.Sequential(torch.nn.Conv2d(self.input_planes,planes,kernel_size=1,stride=stride),
                        torch.nn.BatchNorm2d(planes))
      
      netLayers.append(baseBlock(self.input_planes,planes,stride=stride,shortcut=shortcut))
      self.input_planes = planes
      for i in range(1,num_layers):
          netLayers.append(baseBlock(self.input_planes,planes))
          self.input_planes = planes
      return torch.nn.Sequential(*netLayers)

  def forward(self,x):
      x = F.relu(self.bn1(self.conv1(x)))
      #print("Conv1  :",x.shape)
      x = self.layer1(x)
      #print("L_1:",x.shape)
      x = self.layer2(x)
      #print("L_2:",x.shape)
      x = self.layer3(x)
      #print("L_3:",x.shape)
      x = self.layer4(x)
      #print("L_4:",x.shape)
      x = self.avgPool(x)
      #print("avg:",x.shape)
      x = torch.flatten(x,1)
      #print("flattened:",x.shape)
      out =self.fc(x)
      #print("labels: ",x.shape)
      return x,out

In [ ]:
x= torch.randn(5,3,64,64)
feats,out = ResNet([1,1,1,1],4000,512)(x)
print(out.shape)
print(feats.shape)

torch.Size([5, 4000])
torch.Size([5, 2048])


In [ ]:
def init_weights(m):
  if type(m) == nn.Conv2d:    
    nn.init.kaiming_normal_(m.weight)
    if m.bias is not None:
      nn.init.zeros_(m.bias)
      
  if type(m) == nn.Linear:
    nn.init.xavier_normal_(m.weight)

  if type(m) == nn.BatchNorm2d:
    nn.init.ones_(m.weight)
    nn.init.zeros_(m.bias)

In [ ]:
def save_model(net,epoch):
  current_time = datetime.datetime.now().strftime("%m-%d")
  # Specify a path
  newDir("saved_model")
  PATH = os.getcwd()+f"/saved_model/state_dict_{current_time}_epoch_{epoch}.pt"
  print("----------- saving the model ....\n") 
  torch.save(net.state_dict(), PATH)



def save_output(vector,name):
  current_time = datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S")
  print(" Saving outputs ...")
  newDir('output')
  PATH  = os.getcwd()+"/output/"
  np.save(PATH+ f'{name}_{current_time}.npy',vector)


def visualize(soln_vect,name):
# from HW1_P1
  print(f"Saving graphs for \"{name}\"")
  newDir('output')
  current_time = datetime.datetime.now().strftime("%Y.%m.%d-%H:%M:%S")
  PATH = os.getcwd() + "/output/"
  try :
    plt.plot(soln_vect)
    plt.ylabel(name)
    plt.savefig(PATH +f"{name}_{current_time}.png")
  except Exception as e:
    traceback.print_exc()
    print("Error: Problems generating plots. See if a .png was generated in output folder\n")


class newDir:
  def __init__(self,name):
    self.name = name
    if not os.path.exists(name):
        os.mkdir(name)
        print(f"{self.name} directory created! ")
    else :
        print(f"{self.name} directory exists! ")

class rmDir:
  def __init__(self,name):
    self.name = name
    if os.path.exists(name):
      os.rmdir(name)
      print(f"{self.name} directory is removed! ")
    else :
      print(f"{self.name} directory does not exist! ")


def get_myDataset(data_dir,train=False,val=False,test=False):
  transform_kwargs = {
		'train': transforms.Compose([transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))]),

		'val'  : transforms.Compose([transforms.ToTensor(),
                        transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))]),
	
		'test' : transforms.Compose([transforms.ToTensor(),
                         transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])
			}
  if train:
    root 	  = os.path.join(data_dir,'train_data/')
    transform = transform_kwargs['train']
  if val :
    root      = os.path.join(data_dir,'val_data/')
		#print("root: ",root)
    transform = transform_kwargs['val']
  if test:
    root 	  = os.path.join(data_dir,'test_data/')
    transform = transform_kwargs['test']
  data_set = datasets.ImageFolder(root=root, transform = transform)
  return data_set


def get_data_loader(data_set,**kwargs):
	return torch.utils.data.DataLoader(dataset=data_set,**kwargs)

In [ ]:

def train(model, train_loader, criterion, optimizer):
  model.train()
  running_loss = 0.0
  start_time = time.time()
  for batch_idx, (data, target) in enumerate(train_loader):   
    optimizer.zero_grad()   # .backward() accumulates gradients
    data = data.to(device)
    target = target.to(device) # all data & model on same device

    _,labels = model(data)
    loss = criterion(labels, target)
    running_loss += loss.item()

    loss.backward()
    optimizer.step()
    torch.cuda.empty_cache()
    del data
    del target 
    if (batch_idx+1) % 500 == 0:
      print('Batch_index : {:5d}, Avg Loss in 500 mini batches: {:4.4f}'.format(batch_idx+1, running_loss/500))
      running_loss = 0.0
  return running_loss 

In [ ]:
def test(model, test_loader, criterion):
  with torch.no_grad():
    model.eval()
    running_loss = 0.0
    total_predictions = 0.0
    correct_predictions = 0.0
    start_time = time.time()
    for batch_idx, (data, target) in enumerate(test_loader):   
        data = data.to(device)
        target = target.to(device)
        _,outputs = model(data)
        outputs = F.softmax(outputs,dim=1)
        _,predicted = torch.max(outputs.data, 1)
        total_predictions += target.size(0)
        correct_predictions += (predicted == target).sum().item()
        loss = criterion(outputs, target).detach()
        running_loss += loss.item()
        torch.cuda.empty_cache()
        del data
        del target

  end_time = time.time()
  running_loss /= len(test_loader)
  acc = (correct_predictions/total_predictions)*100.0
  print('Testing Loss: %2.4f'%(running_loss)) 
  print('Testing Accuracy:%2.3f'%(acc), '% ',', Time: %3.3f'%(end_time - start_time), 's')
  return running_loss, acc

In [ ]:
class fetch_image_pairs(torch.utils.data.Dataset):
  def __init__(self,text_pair_path,dir_path,dev_pair=False):
   self.pair_1  = []
   self.pair_2  = []
   self.label   = []
   self.dev_pair = dev_pair
   self.dir_path = dir_path

   with open(text_pair_path) as f:
     for line in f:
       items = line.split()
       self.pair_1.append(items[0])
       self.pair_2.append(items[1])
       if self.dev_pair:
         self.label.append(items[2])
       else:
          self.label.append(-1)

  def __len__(self):
    return len(self.pair_1)
  
  def __getitem__(self,index):
    img1 = Image.open(self.dir_path+self.pair_1[index])
    img2 = Image.open(self.dir_path+self.pair_2[index])
    img1 = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])(img1)
    img2 = transforms.Compose([transforms.ToTensor(),transforms.Normalize((0.485,0.456,0.406),(0.229,0.224,0.225))])(img2)
    lbl = int(self.label[index])
    return img1,img2,lbl


def test_verify(model,vpv_loader):
  sim_score   = []
  exact_score = []
  tart_time = time.time()
  with torch.no_grad():
    model.eval()
    for batch_idx, (img1,img2,true_score) in enumerate(vpv_loader):  
      img1,img2,true_score = img1.to(device), img2.to(device),true_score.to(device)
      embedding_1 = model(img1.float())[0]
      embedding_2 = model(img2.float())[0]
      calc_score = F.cosine_similarity(embedding_1,embedding_2)
      sim_score.append(calc_score.view(-1))
      exact_score.append(true_score.view(-1))
      torch.cuda.empty_cache()
      del true_score
      del img1
      del img2
  end_time = time.time()
  print("Similarity score calculated:!")
  return sim_score,exact_score


In [ ]:
# Hyper parameters
cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")
train_batch_size = 128                      # input batch size for training')
test_batch_size  = 64                      # input batch size for training')
epochs           = 10                       # number of epochs for training
base_lr          = 6.5e-03               # learning rate for a single GPU
lr_cent          = 0.5                      # learning rate for center Loss
weight_cent      = 1                        # Weight of the Center Loss
wd               = 5.0e-04                   # weight decay
num_workers      = 4                        # number of worksers for GPU
momentum         = 0.9                      # SGD momentum
embedding_dim    = 512                      # embedding dimension for images
hidden_layers    = [1,1,1,1]                # ResNET hidden Layers

In [ ]:
data_dir = "/data/classification_data/"; 
dev_set = get_myDataset(data_dir,train=False,val=True,test=False)
kwargs=dict(shuffle=False,batch_size=test_batch_size,num_workers=num_workers,pin_memory=True,drop_last=False)
dev_loader = get_data_loader(dev_set,**kwargs)
print(len(dev_set))

train_set = get_myDataset(data_dir,train=True,val=False,test=False)
kwargs=dict(shuffle=True,batch_size=train_batch_size,num_workers=num_workers,pin_memory=True,drop_last=False)
train_loader = get_data_loader(train_set,**kwargs)
print(len(train_set))

# Now estimating the AUC for the validation pairs
vpv_set    = fetch_image_pairs('/data/verification_pairs_val.txt',"/data/",dev_pair=True)
vpv_loader = DataLoader(vpv_set,batch_size=test_batch_size,shuffle=False,num_workers=num_workers,drop_last=False)

8000
380638


In [ ]:
# Model 
num_classes      = len(train_set.classes)      # number of unique classes
#model = ResNet(hidden_layers,num_classes,embedding_dim)
#model.apply(init_weights)
# Start training from where I left off last time
#PATH  = os.getcwd()+f"/saved_model/state_dict_model_2020.10.22-11:37:35_6.pt"
#model.load_state_dict(torch.load(PATH))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=base_lr,momentum = momentum,weight_decay=wd,nesterov=True)
#scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=3,gamma=0.85)
#scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=.85, last_epoch=-1, verbose=False)
scheduler =torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.85,
                                           patience=2, threshold=0.5, 
                                           threshold_mode='rel', cooldown=0, 
                                           min_lr=0, eps=1e-08, 
                                           verbose=True)
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): baseBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
  )
  (layer2): Sequential(
    (0): baseBlock(
      (conv1): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (bn1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv

In [ ]:
Train_loss = []
Test_loss  = []
Test_acc   = []

print('*-*'*20)
print('train_batch_size: {}, test_batch_size: {}'.format(train_batch_size,test_batch_size))
print('*-*'*20)
print('Epoch : {:2d}, Learning rate : {:.5e} ' .format(1,optimizer.param_groups[0]['lr']))

for epoch in range(epochs):
  train_loss = train(model, train_loader, criterion, optimizer)
  test_loss, test_acc = test_model(model, dev_loader, criterion)
  Train_loss.append(train_loss)
  Test_loss.append(test_loss)
  Test_acc.append(test_acc)
  sim_score,exact_score=test_verify(model,vpv_loader)
  sim_score = torch.cat(sim_score,axis=0)
  exact_score = torch.cat(exact_score,axis=0)
  auc   = roc_auc_score(exact_score.cpu().numpy(),sim_score.cpu().numpy())
  print('='*20)
  print('*auc: {:.3f}'.format(auc))
  print('*^*'*20)
  save_model(model,epoch)
  scheduler.step(test_loss)
  print('Epoch : {:2d}, Learning rate : {:.5e} ' .format(epoch+1,optimizer.param_groups[0]['lr']))
save_output(Train_loss,'Train_loss')
save_output(Test_loss,'Test_loss')
save_output(Test_acc,'Test_acc')
visualize(Train_loss,'Train_loss')
visualize(Test_loss,'Test_loss')
visualize(Test_acc,'Test_acc') 

# Predicting the AUC for the test pairs
vpt_set    = fetch_image_pairs('/data/verification_pairs_test.txt','/data/',dev_pair=False)
vpt_loader = DataLoader(vpt_set,batch_size=test_batch_size,shuffle=False,num_workers=num_workers,drop_last=False)
sim_score,exact_score=test_verify(model,vpt_loader)
sim_score = torch.cat(sim_score,axis=0)


*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
train_batch_size: 128, test_batch_size: 64
*-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-**-*
Epoch :  1, Learning rate : 6.50000e-03 
Batch_index :   500, Avg Loss in 500 mini batches: 0.4348
Batch_index :  1000, Avg Loss in 500 mini batches: 0.5697
Batch_index :  1500, Avg Loss in 500 mini batches: 0.6503


In [ ]:
# preparing the submission file 